# Graph Data Science workshop with Neo4j

Click on the link below to open a Colab version of the notebook. You will be able to create your own version.

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/neo4j-field/graph-summit-apac-2023/blob/main/GDS_Workshop.ipynb" target="_blank">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo">Run your own notebook in Colab
    </a>
  </td>
</table>

---
## Target

Do fraud analysis on a group of persons and transactions using graphs and data science.  

## Context

This notebook allows you load a dataset based on an updated version of [Paysim](https://www.sisu.io/posts/paysim/).  
PaySim is an approximation using an agent-based model and some anonymized, aggregate transactional data from a real mobile money network operator to create synthetic financial data sets academics and hackers can use for exploring ways to detect fraudulent behavior.  
Using this [code](https://github.com/voutilad/paysim), you can generate your own dataset with different caracteristics (size, fraud occurences...).    

We're going to leverage [Neo4j Graph Data Science (GDS)](https://neo4j.com/docs/graph-data-science/current/algorithms/) to investigate through the data and find out fraud patterns and fraudsters.  

## Dataset

The dataset used in this notebook represents money transafers between around 2500 clients, 75 merchants, 5 banks with 175000 transactions across 30 days.  
There are 5 types of transactions:  
* CashIn: a client moves money into the network via a merchant
* CashOut: a client moves money out of the network via a merchant
* Debit: a client moves money into a bank
* Transfer: a client sends money to another client
* Payment: a client exchanges money for something via a merchant

We will try to identify Clients which are fraudsters, trying to target other clients by taking their money below limits ot be unnoticed.  
We added from the the original Paysim some clients details (Phone, Email, SSN) to identify fake profiles too.  

---

## Let's get a graph database

We will use a Neo4j graph database created on the [Neo4j sandbox](https://neo4j.com/sandbox/).  
Once connected, on the _Select a project_ page, go to the section _Your own data_ and select the _Blank Sandbox_.  
Click on the _Create_ button at the bottom of the page.  
After few seconds, you should see the below.  
<img src="./img/sandbox_start.png" alt="Sandbox Start" width="50%" height="50%" title="Sandbox Start">  

And once it's up and running, you can access the connection details by clicking on the top right down arrow and picking the *Connection details* tab.  
You will need 2 things:
* Password  
* Bolt URL   

<img src="./img/sandbox_details.png" alt="Sandbox Details" width="50%" height="50%" title="Sandbox Details">  

---

## Let's code

First we will import the [Neo4j GDS python library](https://pypi.org/project/graphdatascience/)  

In [1]:
# Install Neo4j GDS Python Client
import sys
!{sys.executable} -m pip install graphdatascience

In [2]:
from graphdatascience import GraphDataScience

### Instantiate your GDS Session

Use Neo4j/Bolt URI and credentials according to your setup  

For local standalone instance Bolt connection without auth    
`gds = GraphDataScience("bolt://localhost:7687", auth=None)`  

For local standalone instance Bolt connection with auth    
`gds = GraphDataScience("bolt://localhost:7687", auth=("neo4j", "<password>"))`  

For remote cluster Neo4j connection with auth  
`gds = GraphDataScience("neo4j://<FQDN or IP Address>:7687", auth=("neo4j", "<password>"))`  

For remote standalone instance Bolt connection with auth   
`gds = GraphDataScience("bolt://<FQDN or IP Address>:7687", auth=("neo4j", "<password>"))` 

In [3]:
gds = GraphDataScience("bolt://3.231.156.13:7687", auth=("neo4j", "integrity-crusts-friction")) # >> Update the password and the URL here <<

### Check the GDS version installed

In [4]:
print(f"Neo4j GDS Version: {gds.version()}")

Neo4j GDS Version: 2.3.1


### Optional - Set database if you're not using the default _neo4j_ database. 

Not applicable for AuraDS, Neo4j Desktop and Neo4j Sandbox as we have only one database named _neo4j_.

In [5]:
#gds.set_database("my-db")

### Mark it to True if you want to erase all the data in the db and reload all.
If it is marked False, only the GDS properties will be remove, making the database ready for a rerun.

In [6]:
RELOAD_DATA = True

### Cleaning the database or making it ready for a rerun of the notebook.
Then we will use it to load the data from CSV files, running [Cypher](https://neo4j.com/developer/cypher/) queries.  

In [7]:
if RELOAD_DATA: # Delete all, takes few miniutes on a full database
    gds.run_cypher(
        """
        MATCH (n) CALL {
          WITH n
          DETACH DELETE n
        } IN TRANSACTIONS OF 10 ROWS;
        """
    )
else: # Reset the GDS properties when we re run the book without erasing all
    gds.run_cypher(
        """
        MATCH (c:Client) SET c.fraud_group = null, c.fraud_group_2 = null, c.score = null;
        """
    )

### Loading the data

Using [LOAD CSV](https://neo4j.com/docs/cypher-manual/current/clauses/load-csv/), we are loading csv files into the database, creationg the graph on the fly.  
The first cell is to test the file access, by reading it and showing only the first 5 rows.  

In [8]:
# Checking if we can access the data
if RELOAD_DATA:
    nodeListCSV = gds.run_cypher(
    """
    LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/neo4j-field/graph-summit-apac-2023/main/data/clients.csv" AS row
    RETURN row limit 5
    """
    )
    # The object returned is a Pandas Data Frame, so we can explore using standard Pandas methods
    # The output is not what we expected but remember here were reading the CSV file and loading it into the Data Frame
    nodeListCSV.head(5)

### Creating constraints

For data integrity, we will create [constraints](https://neo4j.com/docs/cypher-manual/current/constraints/) to have a robust graph data model.

In [9]:
if RELOAD_DATA:
    # First we create index
    CONSTRAINTS = [
      "CREATE CONSTRAINT ClientConstraint IF NOT EXISTS FOR (p:Client) REQUIRE p.id IS UNIQUE;",
      "CREATE CONSTRAINT MuleConstraint IF NOT EXISTS FOR (p:Mule) REQUIRE p.id IS UNIQUE;",
      "CREATE CONSTRAINT EmailConstraint IF NOT EXISTS FOR (p:Email) REQUIRE p.email IS UNIQUE;",
      "CREATE CONSTRAINT PhoneConstraint IF NOT EXISTS FOR (p:Phone) REQUIRE p.phoneNumber IS UNIQUE;",
      "CREATE CONSTRAINT SSNConstraint IF NOT EXISTS FOR (p:SSN) REQUIRE p.ssn IS UNIQUE;",
      "CREATE CONSTRAINT MerchantConstraint IF NOT EXISTS FOR (p:Merchant) REQUIRE p.id IS UNIQUE;",
      "CREATE CONSTRAINT BankConstraint IF NOT EXISTS FOR (p:Bank) REQUIRE p.id IS UNIQUE;",
      "CREATE CONSTRAINT TransactionConstraint IF NOT EXISTS FOR (p:Transaction) REQUIRE p.globalStep IS UNIQUE;",
      "CREATE CONSTRAINT DebitConstraint IF NOT EXISTS FOR (p:Transaction) REQUIRE p.globalStep IS UNIQUE;",
      "CREATE CONSTRAINT CashInConstraint IF NOT EXISTS FOR (p:CashIn) REQUIRE p.globalStep IS UNIQUE;",
      "CREATE CONSTRAINT CashOutConstraint IF NOT EXISTS FOR (p:CashOut) REQUIRE p.globalStep IS UNIQUE;",
      "CREATE CONSTRAINT TransferConstraint IF NOT EXISTS FOR (p:Transfer) REQUIRE p.globalStep IS UNIQUE;",
      "CREATE CONSTRAINT PaymentConstraint IF NOT EXISTS FOR (p:Payment) REQUIRE p.globalStep IS UNIQUE;"
    ]
    for c in CONSTRAINTS:
        gds.run_cypher(c)

### Loading all the data

We will load 7 csv files:  
* one for clients   
* one for merchants  
* five for transactions  

We can see how each node is created with a label and at leats one property.  
We see all the relationships between all the nodes, to show the money exchanges betweens all entities.  

In [10]:
if RELOAD_DATA:
    # Load Clients data
    gds.run_cypher(
    """
        LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/neo4j-field/graph-summit-apac-2023/main/data/clients.csv" AS row
        WITH row
        MERGE (c:Client { id: row.ID })
        SET c.name = row.NAME, c.isFraud = toBoolean(row.ISFRAUD)
        MERGE (p:Phone { phoneNumber: row.PHONENUMBER })
        MERGE (c)-[:HAS_PHONE]->(p)
        MERGE (s:SSN { ssn: row.SSN })
        MERGE (c)-[:HAS_SSN]->(s)
        MERGE (e:Email { email: row.EMAIL })
        MERGE (c)-[:HAS_EMAIL]->(e);
    """
    )

In [11]:
if RELOAD_DATA:
    # Load Merchants data
    gds.run_cypher(
    """
        LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/neo4j-field/graph-summit-apac-2023/main/data/merchants.csv" AS row
        WITH row
        MERGE (m:Merchant { id: row.ID })
        SET m.name = row.NAME, m.highRisk = toBoolean(row.HIGHRISK);
    """
    )

In [12]:
if RELOAD_DATA:
    # Load Debit data
    gds.run_cypher(
    """
        LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/neo4j-field/graph-summit-apac-2023/main/data/debit.csv" AS row
        WITH row
        MERGE (b:Bank { id: row.IDDEST })
        SET b.name = row.NAMEDEST
        MERGE (c:Client { id: row.IDORIG })
        MERGE (t:Transaction:Debit { globalStep: row.GLOBALSTEP })
        SET t.amount = toFloat(row.AMOUNT), t.isFraud = toBoolean(row.ISFRAUD)
        MERGE (t)-[:TO]->(b)
        MERGE (c)-[:PERFORMED]->(t);
    """
    )

In [13]:
if RELOAD_DATA:
    # Load CashIn data, largest file then takes few seconds
    gds.run_cypher(
    """
        LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/neo4j-field/graph-summit-apac-2023/main/data/cashin.csv" AS row
        CALL {
            WITH row
            MERGE (m:Merchant { id: row.IDDEST })
            SET m.name = row.NAMEDEST
            MERGE (c:Client { id: row.IDORIG })
            MERGE (t:Transaction:CashIn { globalStep: row.GLOBALSTEP })
            SET t.amount = toFloat(row.AMOUNT), t.isFraud = toBoolean(row.ISFRAUD)
            MERGE (t)-[:TO]->(m)
            MERGE (c)-[:PERFORMED]->(t)
        } IN TRANSACTIONS OF 10 ROWS;
    """
    )

In [14]:
if RELOAD_DATA:
    # Load CashOut data
    gds.run_cypher(
    """
        LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/neo4j-field/graph-summit-apac-2023/main/data/cashout.csv" AS row
        CALL {
            WITH row
            MERGE (m:Merchant { id: row.IDDEST })
            SET m.name = row.NAMEDEST
            MERGE (c:Client { id: row.IDORIG })
            SET c.name = row.NAMEORIG
            MERGE (t:Transaction:CashOut { globalStep: row.GLOBALSTEP })
            SET t.amount = toFloat(row.AMOUNT), t.isFraud = toBoolean(row.ISFRAUD)
            MERGE (t)-[:TO]->(m)
            MERGE (c)-[:PERFORMED]->(t)
        } IN TRANSACTIONS OF 10 ROWS;
    """
    )

In [16]:
if RELOAD_DATA:
    # Load Payment data
    gds.run_cypher(
    """
        LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/neo4j-field/graph-summit-apac-2023/main/data/payment.csv" AS row
        CALL {
            WITH row
            MERGE (m:Merchant { id: row.IDDEST })
            SET m.name = row.NAMEDEST
            MERGE (c:Client { id: row.IDORIG })
            SET c.name = row.NAMEORIG
            MERGE (t:Transaction:Payment { globalStep: row.GLOBALSTEP })
            SET t.amount = toFloat(row.AMOUNT), t.isFraud = toBoolean(row.ISFRAUD)
            MERGE (t)-[:TO]->(m)
            MERGE (c)-[:PERFORMED]->(t)
            FOREACH (ignoreMe in CASE WHEN row.TYPEORIG = 'MULE' THEN [1] ELSE [] END | SET c :Mule)
        } IN TRANSACTIONS OF 5 ROWS;
    """
    )

TransientError: {code: Neo.TransientError.General.MemoryPoolOutOfMemoryError} {message: The allocation of an extra 315.4 MiB would use more than the limit 358.4 MiB. Currently using 43.0 MiB. dbms.memory.transaction.total.max threshold reached}

In [ ]:
if RELOAD_DATA:
    # Load Transfer data
    gds.run_cypher(
    """
    LOAD CSV WITH HEADERS FROM "https://raw.githubusercontent.com/neo4j-field/graph-summit-apac-2023/main/data/transfer.csv" AS row
    CALL {
        WITH row
        MERGE (cd:Client { id: row.IDDEST })
        SET cd.name = row.NAMEDEST
        MERGE (co:Client { id: row.IDORIG })
        SET co.name = row.NAMEORIG
        MERGE (t:Transaction:Transfer { globalStep: row.GLOBALSTEP })
        SET t.amount = toFloat(row.AMOUNT), t.isFraud = toBoolean(row.ISFRAUD)
        MERGE (t)-[:TO]->(cd)
        MERGE (co)-[:PERFORMED]->(t)
        FOREACH (ignoreMe in CASE WHEN row.TYPEDEST = 'MULE' THEN [1] ELSE [] END | SET cd :Mule)
        FOREACH (ignoreMe in CASE WHEN row.TYPEORIG = 'MULE' THEN [1] ELSE [] END | SET co :Mule)
    } IN TRANSACTIONS OF 5 ROWS;
    """
    )

Before updating the graph, you can check out the result with this command in Neo4j Browser (accessible from the Sandbox UI).  
`CALL db.schema.visualization`  
to see the data model from the data you have just loaded

---
### Enriching the graph

Using the transaction details, we are able to enrich the model by adding the ordering of the transaction using the global step.

In [ ]:
if RELOAD_DATA:
    # Update data model with new relationships
    gds.run_cypher(
    """
    MATCH (c:Client) with c.id as clientId
    CALL {
        WITH clientId
        MATCH (c:Client {id: clientId})-[:PERFORMED]->(tx:Transaction)
        WITH c, tx ORDER BY tx.globalStep
        WITH c, collect(tx) AS txs
        WITH c, txs, head(txs) AS _start, last(txs) AS _last

        MERGE (c)-[:FIRST_TX]->(_start)
        MERGE (c)-[:LAST_TX]->(_last)
        WITH c, apoc.coll.pairsMin(txs) AS pairs

        UNWIND pairs AS pair
          WITH pair[0] AS a, pair[1] AS b
          MERGE (a)-[n:NEXT]->(b)
    } IN TRANSACTIONS OF 10 ROWS;
    """
    )

### Having a first look at the dataset

In [ ]:
result = gds.run_cypher(
    """
    CALL db.labels() YIELD label
    CALL apoc.cypher.run('MATCH (:`'+label+'`) RETURN count(*) as freq',{}) YIELD value
    WITH label,value.freq AS freq
    CALL apoc.meta.stats() YIELD nodeCount
    WITH *, 10^3 AS scaleFactor, toFloat(freq)/toFloat(nodeCount) AS relFreq
    RETURN label AS nodeLabel,
        freq AS frequency,
        round(relFreq*scaleFactor)/scaleFactor AS relativeFrequency
    ORDER BY freq DESC
    """
)
result

### Let's have a look on how the money is exchanged across entities

In [ ]:
result = gds.run_cypher(
    """
    MATCH (t:Transaction)
    WITH sum(t.amount) AS globalSum, count(t) AS globalCnt
    WITH *, 10^3 AS scaleFactor
    UNWIND ['CashIn', 'CashOut', 'Payment', 'Debit', 'Transfer'] AS txType
        CALL apoc.cypher.run('MATCH (t:' + txType + ') RETURN sum(t.amount) as txAmount, count(t) AS txCnt', {}) YIELD value
        RETURN txType,
            value.txAmount AS TotalMarketValue,
            100 * round(scaleFactor * (toFloat(value.txAmount) / toFloat(globalSum)))/scaleFactor AS `%MarketValue`,
            100 * round(scaleFactor * (toFloat(value.txCnt) / toFloat(globalCnt)))/scaleFactor AS `%MarketTransactions`,
            toInteger(toFloat(value.txAmount) / toFloat(value.txCnt)) AS AvgTransactionValue,
            value.txCnt AS NumberOfTransactions
        ORDER BY `%MarketTransactions` DESC
    """
)
result

---
### Let's do some data science 

Now that our graph is constructed and filled with data, we can start our analysis.  

We will use the [Weakly Connected Components](https://neo4j.com/docs/graph-data-science/current/algorithms/wcc/) algorithm to investigate.  

---

In [ ]:
# My first graph project name to use wcc algorithm
graphName = 'wccGroups'
# We will focus on fraud group above 5 members
fraudGroupMinSize = 4

In [ ]:
# Remove existing projection with the same name, in case of a re run of the notebook
if gds.graph.exists(graphName).exists:
    gds.graph.drop(gds.graph.get(graphName))

### Lets' find first party fraud

By checking connections among clients, we can identify fake profiles and shady clients.  
We will start with a memory estimate of our projection.  

In [ ]:
gds.graph.project.estimate(
    ['Client', 'SSN', 'Email', 'Phone'],     # Nodes to be added in the projection
    ['HAS_SSN', 'HAS_EMAIL', 'HAS_PHONE'])   # Relationships to be added in the projection

### We will create the projection to be used

We will use the variable _projection_ in our notebook. If you want to use it in Neo4j Browser, you can run 

In [ ]:
projection, projectionPandas = gds.graph.project(graphName, ['Client', 'SSN', 'Email', 'Phone'], ['HAS_SSN', 'HAS_EMAIL', 'HAS_PHONE'])
projectionPandas

### Before running the algorithm, we estimate the memory needed to execute it

In [ ]:
gds.wcc.stream.estimate(projection)

### Let's check how the algorithm will find community in the projection

In [ ]:
result = gds.wcc.stream(projection)
result.head(10)

In [ ]:
result.groupby(['componentId']).count().sort_values('nodeId', ascending=False).head(10)

### Let's populate the graph with a new property to represent the community detected by the wcc algorithm.

In [ ]:
result_wcc = gds.wcc.write(projection, writeProperty='fraud_group', minComponentSize=fraudGroupMinSize)
result_wcc

In [ ]:
# Create an index on the new property just created by the wcc algorithm on Clients
gds.run_cypher("CREATE INDEX ClientFraudIndex IF NOT EXISTS FOR (c:Client) on c.fraud_group;")

In [ ]:
# Look at the community created by the algorithm
# We can see the biggest community has 16 elements
result = gds.run_cypher("""
  MATCH (c:Client) WHERE c.fraud_group IS NOT NULL
  WITH c.fraud_group AS groupId, collect(c.id) AS members
  WITH groupId, size(members) AS groupSize
  WITH collect(groupId) AS groupsOfSize, groupSize
  RETURN groupSize, size(groupsOfSize) AS numOfGroups, groupsOfSize as FraudGroupIds
  ORDER BY groupSize DESC;
""")
result.head(10)

# Look at the large community in Neo4j Browser
Run this Cypher query in Neo4j Browser to visualize the two largest communities.  

`MATCH (c:Client)-[r:HAS_PHONE|HAS_EMAIL|HAS_PHONE]->(n) WHERE c.fraud_group IN [5207, 5649] RETURN c,r,n`  
or just to see large groups  
`MATCH (c:Client) WHERE c.fraud_group IS NOT NULL
WITH c.fraud_group AS groupId, collect(c.id) AS members
WITH *, size(members) AS groupSize WHERE groupSize > 12
MATCH p=(c:Client {fraud_group:groupId})-[:HAS_SSN|HAS_EMAIL|HAS_PHONE]->()
RETURN p`

In [ ]:
result = gds.run_cypher("""
  MATCH (c:Client) WHERE c.fraud_group IS NOT NULL
  WITH c.fraud_group AS groupId, collect(c.id) AS members
  WITH groupId, size(members) AS groupSize WHERE groupSize > $gs
  MATCH (:Client {fraud_group:groupId})-[]-(txn)-[]-(d)      
  WHERE d.fraud_group IS NULL
  UNWIND labels(txn) AS txnType                                                 
  RETURN distinct(txnType), count(txnType);
""", params= {'gs': fraudGroupMinSize} )
result

### Let's create a new property to identofy suspect clients

In [ ]:
result = gds.run_cypher("""
  MATCH (c:Client) WHERE c.fraud_group IS NOT NULL
  WITH c.fraud_group AS groupId, collect(c.id) AS members
  WITH groupId, size(members) AS groupSize WHERE groupSize > $gs
  MATCH (c1:Client {fraud_group:groupId})-[]-(t:Transaction)-[]-(c2:Client)     
  WHERE c2.fraud_group IS NULL
  SET c1.suspect = true, c2.suspect = true                                      
  MERGE (c1)-[r:TRANSACTED_WITH]->(c2)                                          
  ON CREATE SET r += t
  RETURN count(r);
""", params= {'gs': fraudGroupMinSize})
result

### We found 3335 suspect transactions, let's investigate them using again wcc

In [ ]:
graphName2 = 'wccGroups2'

In [ ]:
# Remove existing graph with the same name
if gds.graph.exists(graphName2).exists:
    gds.graph.drop(gds.graph.get(graphName2))

### Creating a new projection using only suspect clients

In [ ]:
projection2, projectionPandas2 = gds.graph.project.cypher(graphName2, 
          'MATCH (c:Client {suspect:true}) RETURN id(c) AS id', 
          'MATCH (c1:Client {suspect:true})-[r:TRANSACTED_WITH]->(c2:Client) RETURN id(c1) AS source, id(c2) as target')
projectionPandas2

In [ ]:
result = gds.wcc.write(projection2, writeProperty='fraud_group_2');

In [ ]:
# Create an index on the new property
gds.run_cypher("CREATE INDEX ClientFraud2Index IF NOT EXISTS FOR (c:Client) on c.fraud_group_2;")

In [ ]:
result = gds.run_cypher("""
MATCH (c:Client) WHERE c.fraud_group_2 IS NOT NULL
WITH c.fraud_group_2 AS secondGroupId, collect(c.id) AS members
RETURN secondGroupId, size(members) AS groupSize
ORDER BY groupSize DESC;
""")
result.head(5)

In [ ]:
graphName3 = 'betweenness'

In [ ]:
# Remove existing graph with the same name
if gds.graph.exists(graphName3).exists:
    gds.graph.drop(gds.graph.get(graphName3))

In [ ]:
projection3, projectionPandas3 = gds.graph.project.cypher(graphName3, 
    'MATCH (c:Client) WHERE c.fraud_group_2 IS NOT NULL WITH c.fraud_group_2 AS secondGroupId, collect(c.id) AS members WITH secondGroupId, size(members) AS groupSize ORDER BY groupSize DESC LIMIT 1 WITH secondGroupId MATCH (c:Client {fraud_group_2:secondGroupId})-[r:TRANSACTED_WITH]-(c2:Client) RETURN id(c) AS id',
    'MATCH (c:Client) WHERE c.fraud_group_2 IS NOT NULL WITH c.fraud_group_2 AS secondGroupId, collect(c.id) AS members WITH secondGroupId, size(members) AS groupSize ORDER BY groupSize DESC LIMIT 1 WITH secondGroupId MATCH (c1:Client {fraud_group_2:secondGroupId})-[:TRANSACTED_WITH]-(c2:Client) RETURN id(c1) AS source, id(c2) AS target'
)

In [ ]:
#Graph Projection Stats from Pandas object
print(projectionPandas3)

In [ ]:
result = gds.betweenness.write(projection3, writeProperty='score')
result

In [ ]:
gds.run_cypher("""
  CALL gds.betweenness.stream('betweenness') YIELD nodeId, score
  WITH gds.util.asNode(nodeId) AS c, score WHERE score > 0                       // Filter 0 scores again
  MATCH (c)-[r:TRANSACTED_WITH]-(:Client)                                        // Retrieve the relationships
  WITH c.name AS name, score, collect(r) AS txns                                 // Collect and count the number of relationships
  WITH name, score AS original, score/size(txns) AS newScore
  RETURN name, newScore, original ORDER BY newScore DESC;  
""")